In [1]:
import json
from PIL import Image, ImageDraw, ImageFont
import os, shutil
import numpy as np
import pandas as pd
from matplotlib import colors, cm, pyplot as plt
import glob
import sys

In [2]:
PIXELS_X = 910
PIXELS_Y = 910  # equal to the number of scan lines

In [3]:
predictions_file = '/Users/darylwilding-mcbride/Downloads/experiments/dwm-test/predictions/yolov3-tiny_3l-tile-35-thresh-0_1-predictions.json'

In [4]:
tiles_dir = '/Users/darylwilding-mcbride/Downloads/experiments/dwm-test/tiles/190719_Hela_Ecoli_1to1_01/tile-35'
tiles_with_predictions_dir = '{}/predictions'.format(tiles_dir)
tiles_with_tracked_objects_dir = '{}/tracked-features'.format(tiles_dir)

In [5]:
if os.path.exists(tiles_with_predictions_dir):
    shutil.rmtree(tiles_with_predictions_dir)
os.makedirs(tiles_with_predictions_dir)

In [6]:
if os.path.exists(tiles_with_tracked_objects_dir):
    shutil.rmtree(tiles_with_tracked_objects_dir)
os.makedirs(tiles_with_tracked_objects_dir)

In [7]:
with open(predictions_file) as file:
    predictions = json.load(file)

In [8]:
feature_label_font = ImageFont.truetype('/Library/Fonts/Arial.ttf', 10)

In [9]:
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]

In [10]:
def calculate_iou(bb_test, bb_gt):
    # Computes IUO between two bounding boxes in the form [x1,y1,x2,y2]
    # Source: https://github.com/abewley/sort
    xx1 = np.maximum(bb_test[0], bb_gt[0])
    yy1 = np.maximum(bb_test[1], bb_gt[1])
    xx2 = np.minimum(bb_test[2], bb_gt[2])
    yy2 = np.minimum(bb_test[3], bb_gt[3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / ((bb_test[2]-bb_test[0])*(bb_test[3]-bb_test[1]) + (bb_gt[2]-bb_gt[0])*(bb_gt[3]-bb_gt[1]) - wh)
    return(o)


In [11]:
MINIMUM_IOU_FOR_ASSOCIATION = 0.4

In [12]:
detected_objects = []
feature_id = 0
for frame in predictions:
    prediction_frame_id = frame['frame_id']
    tile_name = os.path.basename(frame['filename'])
    source_tile_name = '{}/{}'.format(tiles_dir, tile_name)
    predictions_tile_name = '{}/{}'.format(tiles_with_predictions_dir, tile_name)
    tracked_objects_tile_name = '{}/{}'.format(tiles_with_tracked_objects_dir, tile_name)
    
    # load the source tile
    tile_predictions_img = Image.open(source_tile_name)
    draw_prediction = ImageDraw.Draw(tile_predictions_img)
    
    object_id = 1
    for prediction in frame['objects']:
        class_id = prediction['class_id']
        class_name = prediction['name']
        bounding_box = prediction['relative_coordinates']
        confidence = prediction['confidence']
        
        # calculate the coordinates
        x1 = (bounding_box['center_x'] - (bounding_box['width'] / 2)) * PIXELS_X
        y1 = (bounding_box['center_y'] - (bounding_box['height'] / 2)) * PIXELS_Y
        w = bounding_box['width'] * PIXELS_X
        h = bounding_box['height'] * PIXELS_Y
        x2 = x1 + w
        y2 = y1 + h

        # draw the prediction
        draw_prediction.rectangle(xy=[(x1, y1), (x2, y2)], fill=None, outline=(100,255,100,20))
        draw_prediction.text((x1, y1-12), '{} ({})'.format(class_name, round(confidence,1)), font=feature_label_font, fill=(100,255,100,20))
        
        # add this prediction to the list of objects to track
        detected_objects.append((prediction_frame_id, object_id, class_id, x1, y1, x2, y2, confidence, feature_id))
        
        object_id += 1

    # save the tile with the predictions overlaid
    tile_predictions_img.save(predictions_tile_name)

In [54]:
detected_objects_df = pd.DataFrame(detected_objects, columns=['prediction_frame_id','object_id','class_id','x1','y1','x2','y2','confidence','feature_id'])

In [55]:
detected_objects_df.head()

,prediction_frame_id,object_id,class_id,x1,y1,x2,y2,confidence,feature_id
0,1,1,0,678.166125,625.382485,806.400595,671.824335,1.000000,0
1,1,2,0,429.943605,655.831540,487.733155,703.662960,0.999964,0
2,1,3,0,822.008915,614.897465,919.814805,664.143935,0.999950,0
3,1,4,0,562.693950,481.818155,632.194290,508.340105,0.999898,0
4,1,5,0,138.911500,608.286770,277.919460,693.479150,0.905883,0


In [56]:
objects_frame_2 = detected_objects_df[detected_objects_df.prediction_frame_id == 2]
objects_frame_1 = detected_objects_df[detected_objects_df.prediction_frame_id == 1]

In [57]:
len(objects_frame_1), len(objects_frame_2)

(9, 8)

In [59]:
# find in these two DFs the objects that belong to the same feature
frame_1 = 1
frame_2 = 2
associations = []
for frame_1_index, frame_1_row in objects_frame_1.iterrows():
    object_id_1 = int(frame_1_row[1])
    for frame_2_index, frame_2_row in objects_frame_2.iterrows():
        object_id_2 = int(frame_2_row[1])
        bb_1 = [frame_1_row[3], frame_1_row[4], frame_1_row[5], frame_1_row[6]]
        bb_2 = [frame_2_row[3], frame_2_row[4], frame_2_row[5], frame_2_row[6]]
        iou = calculate_iou(bb_1, bb_2)
        if iou >= MINIMUM_IOU_FOR_ASSOCIATION:
            associations.append((frame_1, object_id_1, frame_2, object_id_2, iou))
        

In [60]:
associations

[(1, 1, 2, 6, 0.48962615017039773),
 (1, 3, 2, 5, 0.7168611439911589),
 (1, 4, 2, 2, 0.6591962683273683),
 (1, 6, 2, 3, 0.6166415146886148),
 (1, 9, 2, 7, 0.43541329889587566)]

In [61]:
feature_id = 1
for association in associations:
    # check if the associated object already has a feature_id - if so, we should use it
    frame_1_feature_id = int(detected_objects_df[(detected_objects_df.prediction_frame_id == association[0]) & (detected_objects_df.object_id == association[1])].iloc[0].feature_id)
    if frame_1_feature_id == 0:
        detected_objects_df.loc[(detected_objects_df.prediction_frame_id == association[0]) & (detected_objects_df.object_id == association[1]), 'feature_id'] = feature_id
        detected_objects_df.loc[(detected_objects_df.prediction_frame_id == association[2]) & (detected_objects_df.object_id == association[3]), 'feature_id'] = feature_id
        feature_id += 1
    else:
        detected_objects_df.loc[(detected_objects_df.prediction_frame_id == association[2]) & (detected_objects_df.object_id == association[3]), 'feature_id'] = frame_1_feature_id
        

In [62]:
detected_objects_df

,prediction_frame_id,object_id,class_id,x1,y1,x2,y2,confidence,feature_id
0,1,1,0,678.166125,625.382485,806.400595,671.824335,1.000000,1
1,1,2,0,429.943605,655.831540,487.733155,703.662960,0.999964,0
2,1,3,0,822.008915,614.897465,919.814805,664.143935,0.999950,2
3,1,4,0,562.693950,481.818155,632.194290,508.340105,0.999898,3
4,1,5,0,138.911500,608.286770,277.919460,693.479150,0.905883,0
5,1,6,0,529.979450,644.272265,620.244170,694.397795,0.681667,4
6,1,7,0,51.714845,635.590865,104.947115,709.461935,0.635575,0
7,1,8,0,153.464220,627.711630,220.766000,678.176590,0.095396,0
8,1,9,0,36.546965,611.540930,123.480175,678.422290,0.020823,5
9,2,1,0,156.350285,861.327740,214.718595,895.827660,0.999996,0
